In [1]:
import pymongo
from pymongo import MongoClient

In [2]:
import pandas as pd
import numpy as np
import math
import json
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor, AdaBoostClassifier, AdaBoostRegressor 
from sklearn.svm import LinearSVC, LinearSVR, SVC, SVR
from xgboost import XGBClassifier
from sklearn.linear_model import LinearRegression

% matplotlib inline

UsageError: Line magic function `%` not found.


In [3]:
#Connecting to mongodb client
client = MongoClient('localhost',27017)
db = client.starbucks2
data = db.str_Data
df = pd.DataFrame(list(data.find()))
df

,_id,offer_id,event,customer_id,time,amount,reward,F,M,O,...,bogo,discount,informational,email,mobile,social,web,duration,difficulty,cust_resp
0,632a9aa20d67f76bfdc5f9d7,4d5c57ea9a6940dd891ad53e9dbe8da0,offer received,0020c2b971eb4e9188eac86d93036a77,408,0.0,0,1,0,0,...,1,0,0,1,1,1,1,120.0,10,1
1,632a9aa20d67f76bfdc5f9d8,4d5c57ea9a6940dd891ad53e9dbe8da0,offer viewed,0020c2b971eb4e9188eac86d93036a77,426,0.0,0,1,0,0,...,1,0,0,1,1,1,1,120.0,10,1
2,632a9aa20d67f76bfdc5f9d9,ae264e3637204a6fb9bb56bc8210ddfd,offer received,0020c2b971eb4e9188eac86d93036a77,168,0.0,0,1,0,0,...,1,0,0,1,1,1,0,168.0,10,1
3,632a9aa20d67f76bfdc5f9da,fafdcd668e3743c1bb461111dcafc2a4,offer received,0020c2b971eb4e9188eac86d93036a77,0,0.0,0,1,0,0,...,0,1,0,1,1,1,1,240.0,10,1
4,632a9aa20d67f76bfdc5f9db,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,0020c2b971eb4e9188eac86d93036a77,12,0.0,0,1,0,0,...,0,1,0,1,1,1,1,240.0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116356,632a9aa40d67f76bfdc7b25b,ae264e3637204a6fb9bb56bc8210ddfd,offer viewed,ffaecba5bbb14cffb04c2490d18eee7e,510,0.0,0,0,1,0,...,1,0,0,1,1,1,0,168.0,10,1
116357,632a9aa40d67f76bfdc7b25c,fafdcd668e3743c1bb461111dcafc2a4,offer received,ffaecba5bbb14cffb04c2490d18eee7e,576,0.0,0,0,1,0,...,0,1,0,1,1,1,1,240.0,10,1
116358,632a9aa40d67f76bfdc7b25d,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,ffaecba5bbb14cffb04c2490d18eee7e,600,0.0,0,0,1,0,...,0,1,0,1,1,1,1,240.0,10,1
116359,632a9aa40d67f76bfdc7b25e,3f207df678b143eea3cee63160fa8bed,offer received,ffaecba5bbb14cffb04c2490d18eee7e,168,0.0,0,0,1,0,...,0,0,1,1,1,0,1,96.0,0,1


In [4]:
df.drop(columns=['_id','customer_id', 'offer_id', 'time', 'email','event'], inplace=True)
df.head()

,amount,reward,F,M,O,1,2,3,4,5,...,age_90,bogo,discount,informational,mobile,social,web,duration,difficulty,cust_resp
0,0.0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,1,1,1,120.0,10,1
1,0.0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,1,1,1,120.0,10,1
2,0.0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,1,1,0,168.0,10,1
3,0.0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,1,1,1,240.0,10,1
4,0.0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,1,1,1,240.0,10,1


In [5]:
X = df.drop(columns=['cust_resp'])
y =df.loc[:,['cust_resp']]

In [6]:
X

,amount,reward,F,M,O,1,2,3,4,5,...,age_80,age_90,bogo,discount,informational,mobile,social,web,duration,difficulty
0,0.0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,1,1,1,120.0,10
1,0.0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,1,1,1,120.0,10
2,0.0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,1,1,0,168.0,10
3,0.0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,1,1,1,240.0,10
4,0.0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,1,1,1,240.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116356,0.0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,1,0,168.0,10
116357,0.0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,1,1,240.0,10
116358,0.0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,1,1,240.0,10
116359,0.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,96.0,0


In [7]:
y

,cust_resp
0,1
1,1
2,1
3,1
4,1
...,...
116356,1
116357,1
116358,1
116359,1


In [8]:
# Distribution of target class
y.squeeze().value_counts()

1    78028
0    38333
Name: cust_resp, dtype: int64

In [9]:
#Split features and labels into train & te #st set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
# Prepare list of features to scale
feat_to_scale = ['difficulty', 'duration', 'reward', 'amount']

In [11]:
def func_feat_scale(df, feat=feat_to_scale):

   
        # Prepare dataframe of only features to scale
        df_feat_scale = df.loc[:,feat]
        
        # Apply feature scaling to df
        scaler = MinMaxScaler()
        df_feat_scale = pd.DataFrame(scaler.fit_transform(df_feat_scale), columns = df_feat_scale.columns,\
                                     index=df_feat_scale.index)
        
        # Drop orignal features to scale from df and add scaled features back to df
        df = df.drop(columns=feat, axis=1)
        df_scaled = pd.concat([df, df_feat_scale], axis=1)
        
        return df_scaled

In [12]:
X_train_scaled = func_feat_scale(X_train, feat=feat_to_scale)
X_train_scaled.head()

,F,M,O,1,2,3,4,5,6,7,...,bogo,discount,informational,mobile,social,web,difficulty,duration,reward,amount
88126,0,1,0,0,0,0,0,0,0,1,...,1,0,0,1,1,1,0.25,0.285714,0.0,0.0
71937,1,0,0,0,0,0,1,0,0,0,...,0,1,0,1,1,1,0.35,0.571429,0.0,0.0
64460,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0.00,0.000000,0.0,0.0
92424,1,0,0,0,0,0,1,0,0,0,...,0,0,1,1,1,0,0.00,0.000000,0.0,0.0
25866,0,1,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,0.35,0.571429,0.0,0.0


In [13]:
# Print shape of dataframe X_train_scaled
X_train_scaled.shape

(87270, 51)

In [14]:
# Print shape of y_train
y_train.shape

(87270, 1)

In [15]:
# Define fit_classifier function to fit different classifiers on training dataset i.e. X_train_scaled
# to find the best performing classifier algorithm
def fit_classifier(clf, param_grid, X=X_train_scaled.values, y=y_train.squeeze().values):
    
    
    # cv uses StratifiedKFold
    # scoring f1 available as parameter
    start = time.time()
    grid = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='f1', cv=5, verbose=0)
    print("Training {} :".format(clf.__class__.__name__))
    grid.fit(X, y)
    end = time.time()
    time_taken = round(end-start,2)

    print(clf.__class__.__name__)
    print("Time taken : {} secs".format(time_taken))
    print("Best f1_score : {}".format(round(grid.best_score_,4)))
    print("*"*40)
    
    return grid.best_score_, grid.best_estimator_, time_taken

In [16]:
# Initialize classification algorithms
lor = LogisticRegression(random_state=42) # LogisticRegression
rfc = RandomForestClassifier(random_state=42) # RandomForestClassifier
abc = AdaBoostClassifier(random_state=42) # AdaBoostClassifier
gbc = GradientBoostingClassifier(random_state=42) # GradientBoostingClassifier

In [17]:
# Find best classification algorithm
clf_names = []
clf_scores = []
clf_best_ests = []
clf_time_taken = []
clf_dict = {}

for clf in [lor, rfc, abc, gbc]:
    best_score, best_est, time_taken = fit_classifier(clf, {})
    clf_names.append(clf.__class__.__name__)
    clf_scores.append(best_score)
    clf_best_ests.append(best_est)
    clf_time_taken.append(time_taken)

Training LogisticRegression :
LogisticRegression
Time taken : 2.63 secs
Best f1_score : 0.7986
****************************************
Training RandomForestClassifier :
RandomForestClassifier
Time taken : 49.05 secs
Best f1_score : 0.8398
****************************************
Training AdaBoostClassifier :
AdaBoostClassifier
Time taken : 22.66 secs
Best f1_score : 0.799
****************************************
Training GradientBoostingClassifier :
GradientBoostingClassifier
Time taken : 66.27 secs
Best f1_score : 0.8008
****************************************


In [18]:
# Create clf_df dataframe from clf_dict
clf_dict['best_f1_score'] = clf_scores
clf_dict['time_taken(s)'] = clf_time_taken
clf_dict['best_est'] = clf_best_ests
clf_df = pd.DataFrame(clf_dict, index=clf_names)
clf_df

,best_f1_score,time_taken(s),best_est
LogisticRegression,0.798623,2.63,LogisticRegression(random_state=42)
RandomForestClassifier,0.839814,49.05,"(DecisionTreeClassifier(max_features='sqrt', r..."
AdaBoostClassifier,0.798998,22.66,"(DecisionTreeClassifier(max_depth=1, random_st..."
GradientBoostingClassifier,0.800753,66.27,([DecisionTreeRegressor(criterion='friedman_ms...


In [19]:
# Print the best estimator of RandomForestClassifier
clf_df.loc['RandomForestClassifier', 'best_est']


RandomForestClassifier(random_state=42)

In [20]:
# Tune the best classifier(RandomForestClassifier) with the help of param grid in GridSearchCV
# The fine tuned model will be used with the test set
param_grid = {
              'n_estimators': [50],
              
              'max_depth': [None],
              
              'min_samples_split': [3],
              
              'min_samples_leaf': [1]
             }

rfc = RandomForestClassifier(random_state=42)
rfc_best_score, rfc_best_est, _ = fit_classifier(rfc,param_grid)
rfc_best_est

Training RandomForestClassifier :
RandomForestClassifier
Time taken : 24.04 secs
Best f1_score : 0.84
****************************************


RandomForestClassifier(min_samples_split=3, n_estimators=50, random_state=42)

In [21]:
# Prepare test data in order to predict classes for the test data
# Scale selected features in test set i.e. X_test
X_test_scaled = func_feat_scale(X_test, feat=feat_to_scale)
X_test_scaled.head()

,F,M,O,1,2,3,4,5,6,7,...,bogo,discount,informational,mobile,social,web,difficulty,duration,reward,amount
72215,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0.00,0.000000,0.0,0.0
23869,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,1,0.25,0.285714,0.0,0.0
77837,0,1,0,0,0,0,0,1,0,0,...,0,0,1,1,1,0,0.00,0.000000,0.0,0.0
98568,0,1,0,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0.00,0.000000,0.0,0.0
111314,0,1,0,0,0,0,1,0,0,0,...,0,1,0,1,1,1,0.50,1.000000,0.0,0.0


In [22]:
# Print shape of dataframe X_test_scaled
X_test_scaled.shape

(29091, 51)

In [23]:
# Print shape of y_test
y_test.shape

(29091, 1)

In [24]:
# Classification of test data using best model trained on train data
y_pred = rfc_best_est.predict(X_test_scaled)
y_pred

C:\Users\acer\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [25]:
# Calculate tn, fp, fn, tp from confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test.values, y_pred).ravel()
print('true negatives: {}'.format(tn))
print('false postives: {}'.format(fp))
print('false negatives: {}'.format(fn))
print('true postives: {}'.format(tp))

true negatives: 6111
false postives: 3425
false negatives: 2694
true postives: 16861


In [26]:
## Calculate precision, recall and f1_score of the test data
test_data_f1_score = f1_score(y_test.squeeze().values, y_pred)
test_data_precision_score = precision_score(y_test.squeeze().values.tolist(), y_pred)
test_data_recall_score = recall_score(y_test.squeeze().values, y_pred)
print('f1_score on test data: {}'.format(round(test_data_f1_score,4)))
print('precision score on test data: {}'.format(round(test_data_precision_score,4)))
print('recall score on test data: {}'.format(round(test_data_recall_score,4)))

f1_score on test data: 0.8464
precision score on test data: 0.8312
recall score on test data: 0.8622
